In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt

# Useful Info For loading Data
https://www.kaggle.com/dimarudov/data-analysis-using-sql

Below we connect to the database

In [2]:

path = "../input/"  #Insert path here
database = path + 'database.sqlite'

conn = sqlite3.connect(database)

We load att the games with home goal, away goal and betting odds for different companies

In [3]:

detailed_bets = pd.read_sql("""SELECT home_team_goal, 
                                        away_team_goal,
                                        B365H,
                                        B365D,
                                        B365A,
                                        BWH,
                                        BWD,
                                        BWA,
                                        IWH,
                                        IWD,
                                        IWA,
                                        LBH,
                                        LBD,
                                        LBA,
                                        PSH,
                                        PSD,
                                        PSA,
                                        WHH,
                                        WHD,
                                        WHA,
                                        SJH,
                                        SJD,
                                        SJA,
                                        VCH,
                                        VCD,
                                        VCA,
                                        GBH,
                                        GBD,
                                        GBA,
                                        BSH,
                                        BSD,
                                        BSA
                                        
                                        
                                FROM Match;""", conn)

detailed_bets.shape

(25979, 32)

We check all the columns and see if there are lots of NaNs, we noticed that certain betting companies have lots of them

In [4]:
detailed_bets.isna().sum()

home_team_goal        0
away_team_goal        0
B365H              3387
B365D              3387
B365A              3387
BWH                3404
BWD                3404
BWA                3404
IWH                3459
IWD                3459
IWA                3459
LBH                3423
LBD                3423
LBA                3423
PSH               14811
PSD               14811
PSA               14811
WHH                3408
WHD                3408
WHA                3408
SJH                8882
SJD                8882
SJA                8882
VCH                3411
VCD                3411
VCA                3411
GBH               11817
GBD               11817
GBA               11817
BSH               11818
BSD               11818
BSA               11818
dtype: int64

We are dropping GB, BS, and PS betting companies because they have lots of nulls

In [5]:
detailed_bets_filtered = pd.read_sql("""SELECT home_team_goal, 
                                        away_team_goal,
                                        B365H,
                                        B365D,
                                        B365A,
                                        BWH,
                                        BWD,
                                        BWA,
                                        IWH,
                                        IWD,
                                        IWA,
                                        LBH,
                                        LBD,
                                        LBA,
                                        WHH,
                                        WHD,
                                        WHA,
                                        SJH,
                                        SJD,
                                        SJA,
                                        VCH,
                                        VCD,
                                        VCA
                                        
                                        
                                FROM Match;""", conn)

We drop all the matches that have lots of naans and get 17 000 games

In [7]:
cleaned_bets = detailed_bets_filtered.dropna(thresh=23) #drops vals if there are more than 1 Nan

cleaned_bets.shape

(16980, 23)

In [36]:
detailed_bets['home_win'] = 0
detailed_bets.loc[detailed_bets['home_team_goal'] < detailed_bets['away_team_goal'], 'home_win'] = 1
detailed_bets.loc[detailed_bets['home_team_goal'] > detailed_bets['away_team_goal'], 'home_win'] = -1
detailed_bets


home_team_goal  away_team_goal  B365H  B365D  B365A   BWH   BWD    BWA  \
0                   1               1   1.73   3.40   5.00  1.75  3.35   4.20   
1                   0               0   1.95   3.20   3.60  1.80  3.30   3.95   
2                   0               3   2.38   3.30   2.75  2.40  3.30   2.55   
3                   5               0   1.44   3.75   7.50  1.40  4.00   6.80   
4                   1               3   5.00   3.50   1.65  5.00  3.50   1.60   
5                   1               1   4.75   3.40   1.67  4.85  3.40   1.65   
6                   2               2   2.10   3.20   3.30  2.05  3.25   3.15   
7                   1               2   3.20   3.40   2.20  2.55  3.30   2.40   
8                   1               0   2.25   3.25   2.88  2.30  3.25   2.70   
9                   4               1   1.30   5.25   9.50  1.25  5.00  10.00   
10                  1               2   1.30   5.25   9.50  1.25  5.00  10.00   
11                  0               2   2.60   3.20   2.50  2.60  3.20   2.40   
12                  0               0   1.91   3.40   4.00  1.85  3.35   3.80   
13                  2               2   2.90   3.30   2.38  2.75  3.30   2.25   
14                  1               2   1.70   3.30   4.50  1.75  3.45   4.00   
15                  0               1   2.35   3.30   3.00  2.35  3.25   2.70   
16                  1               3   6.00   3.75   1.57  5.10  3.60   1.55   
17                  1               3   4.00   3.25   1.83  4.30  3.50   1.68   
18                  2               3   1.36   4.75   8.50  1.25  5.00   9.00   
19                  0               0   2.40   3.30   2.90  2.25  3.20   2.80   
20                  2               2   3.20   3.30   2.25  3.25  3.20   2.10   
21                  2               0   1.75   3.75   4.33  1.85  3.35   3.65   
22                  1               1   1.40   4.33   8.00  1.38  4.00   7.20   
23                  1               2   1.83   3.50   4.20  1.80  3.35   3.90   
24                  0               0   2.00   3.50   3.60  1.95  3.25   3.40   
25                  1               0   2.70   3.25   2.60  2.50  3.30   2.45   
26                  1               3   3.60   3.30   2.05  3.40  3.25   1.95   
27                  1               1   2.30   3.25   2.90  2.20  3.15   3.00   
28                  1               1   1.25   5.25  10.00  1.23  5.00  10.00   
29                  2               2   2.00   3.25   3.50  2.10  3.20   3.20   
...               ...             ...    ...    ...    ...   ...   ...    ...   
25949               6               1    NaN    NaN    NaN   NaN   NaN    NaN   
25950               1               3    NaN    NaN    NaN   NaN   NaN    NaN   
25951               0               2    NaN    NaN    NaN   NaN   NaN    NaN   
25952               1               1    NaN    NaN    NaN   NaN   NaN    NaN   
25953               3               1    NaN    NaN    NaN   NaN   NaN    NaN   
25954               3               1    NaN    NaN    NaN   NaN   NaN    NaN   
25955               0               1    NaN    NaN    NaN   NaN   NaN    NaN   
25956               0               1    NaN    NaN    NaN   NaN   NaN    NaN   
25957               3               1    NaN    NaN    NaN   NaN   NaN    NaN   
25958               3               3    NaN    NaN    NaN   NaN   NaN    NaN   
25959               1               3    NaN    NaN    NaN   NaN   NaN    NaN   
25960               1               3    NaN    NaN    NaN   NaN   NaN    NaN   
25961               0               2    NaN    NaN    NaN   NaN   NaN    NaN   
25962               1               0    NaN    NaN    NaN   NaN   NaN    NaN   
25963               2               5    NaN    NaN    NaN   NaN   NaN    NaN   
25964               0               1    NaN    NaN    NaN   NaN   NaN    NaN   
25965               1               0    NaN    NaN    NaN   NaN   NaN    NaN   
25966               0               2 